### Brooklyn Travel Demand Model - V2

This notebook implements a methodologically sound workflow to generate a mode-agnostic travel demand matrix for Brooklyn. It addresses the flaws in the previous `multimodal.ipynb` notebook by:

1.  **Using a Proper Zonal System:** Employs Census Block Groups for Brooklyn instead of a coarse grid.
2.  **Generating Mode-Agnostic Demand:** Calculates total daily person-trips first, before considering mode choice.
3.  **Balancing with Control Totals:** Scales trip productions and attractions to match observed totals from the NYC Citywide Mobility Survey (CMS).
4.  **Calibrating the Gravity Model:** Calibrates the gravity model's friction factor to match the observed average trip length from the CMS data.

This workflow uses the `grid2demand` library where possible and supplements it with custom utility functions for zone preparation, balancing, and calibration.

In [3]:
# Step 0: Setup and Imports

import os
import pandas as pd
import sys

# Add utils to path to import custom modules
utils_path = os.path.abspath('utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)

# Add grid2demand to path
grid2demand_path = os.path.abspath('src/tdm')
if grid2demand_path not in sys.path:
    sys.path.append(grid2demand_path)

import grid2demand as gd
from utils import zones, calibration

import warnings
warnings.filterwarnings('ignore')

# Define constants and file paths
INPUT_DIR = 'input_data'
DATA_DIR = 'data' # For intermediate and output files
SETTINGS_DIR = 'settings'
os.makedirs(DATA_DIR, exist_ok=True)

BROOKLYN_FIPS = '047' # FIPS code for Kings County
ZONE_ID_COL = 'GEOID20' # Unique ID for census blocks

CENSUS_SHP_PATH = os.path.join(INPUT_DIR, 'census/ny_state_blocks/tl_2022_36_tabblock20.shp')
ZONE_CSV_PATH = os.path.join(DATA_DIR, 'zone_brooklyn_census_blocks.csv')

TRIP_RATE_INPUT_PATH = os.path.join(SETTINGS_DIR, 'brooklyn_poi_trip_rate_all_purposes.csv')
TRIP_RATE_TOTAL_PATH = os.path.join(SETTINGS_DIR, 'brooklyn_poi_trip_rate_total_daily.csv')

CMS_TRIP_PATH = os.path.join(INPUT_DIR, 'cms/Citywide_Mobility_Survey_-_Trip_2019.csv')
CMS_HH_PATH = os.path.join(INPUT_DIR, 'cms/Citywide_Mobility_Survey_-_Household_2019.csv')

print("Setup complete. Paths are defined.")

Setup complete. Paths are defined.


#### Step 1: Prepare Zonal System

Convert the Census Block shapefile for Brooklyn into a `zone.csv` file that `grid2demand` can read.

In [5]:
# Step 1: Preparing Zonal System

try:
    zones.shapefile_to_zone_csv(
        shapefile_path=CENSUS_SHP_PATH,
        output_path=ZONE_CSV_PATH,
        zone_id_col=ZONE_ID_COL,
        county_fp=BROOKLYN_FIPS
    )
    print(f"Successfully created zone file at: {ZONE_CSV_PATH}")
except Exception as e:
    print(f"An error occurred during zone preparation: {e}")

Reading shapefile from input_data/census/ny_state_blocks/tl_2022_36_tabblock20.shp...
  Initial shapefile has 288819 features.
  Filtered to 9855 features for county 047.
Reprojecting to EPSG:2263...
Saving zone.csv to data/zone_brooklyn_census_blocks.csv...
  Done.
Successfully created zone file at: data/zone_brooklyn_census_blocks.csv


#### Step 2: Prepare Mode-Agnostic Trip Rates

To generate total daily demand, we need to aggregate the trip rates across all purposes (HBW, HBO, NHB).

In [6]:
# Step 2: Preparing Mode-Agnostic Trip Rates

try:
    trip_rates_df = pd.read_csv(TRIP_RATE_INPUT_PATH)
    
    # Group by POI type and sum rates across all purposes
    total_rates = trip_rates_df.groupby(['poi_type_id', 'building', 'unit_of_measure']).agg({
        'production_rate': 'sum',
        'attraction_rate': 'sum'
    }).reset_index()
    
    # Add a dummy trip_purpose column (e.g., purpose 4 for 'total') as grid2demand expects it
    total_rates['trip_purpose'] = 4 # Representing 'total daily'
    
    total_rates.to_csv(TRIP_RATE_TOTAL_PATH, index=False)
    print(f"Successfully created total daily trip rate file at: {TRIP_RATE_TOTAL_PATH}")
    print("Total Daily Trip Rates Head:")
    print(total_rates.head())
except Exception as e:
    print(f"An error occurred during trip rate preparation: {e}")

Successfully created total daily trip rate file at: settings/brooklyn_poi_trip_rate_total_daily.csv
Total Daily Trip Rates Head:
   poi_type_id     building    unit_of_measure  production_rate  \
0            0       office  1,000 Sq. Ft. GFA             4.51   
1            1  residential  1,000 Sq. Ft. GFA             1.45   
2            2   apartments  1,000 Sq. Ft. GFA             1.45   
3            3        house  1,000 Sq. Ft. GFA             1.85   
4            4        hotel           per room             5.34   

   attraction_rate  trip_purpose  
0            20.59             4  
1             4.12             4  
2             4.12             4  
3             5.32             4  
4            33.06             4  


#### Step 3: Initialize `grid2demand` and Load Data

In [ ]:
print("
--- Step 3: Initializing grid2demand and Loading Data ---")
# We use the 'data' directory as the main input/output for grid2demand files
net = gd.GRID2DEMAND(input_dir=DATA_DIR)

# Load network (node.csv, poi.csv)
print("Loading network...")
net.load_network()

# Load our new census-based zones
print("Loading zones from CSV...")
net.taz2zone(zone_file=ZONE_CSV_PATH)

# Map nodes and POIs to the new zones
print("Mapping nodes and POIs to zones...")
net.map_zone_node_poi()

print("Data loading complete.")

#### Step 4: Generate and Balance Productions & Attractions

In [ ]:
print("
--- Step 4: Generating and Balancing P's & A's ---")

# Calculate initial productions and attractions using the total daily rates
print("Calculating initial productions and attractions...")
net.calc_zone_prod_attr(
    trip_rate_file=TRIP_RATE_TOTAL_PATH,
    trip_purpose=4 # Corresponds to the dummy purpose in our total rates file
)

initial_p = sum(z.get('production', 0) for z in net.zone_dict.values())
initial_a = sum(z.get('attraction', 0) for z in net.zone_dict.values())
print(f"  Initial total productions: {initial_p:,.0f}")
print(f"  Initial total attractions: {initial_a:,.0f}")

# Load CMS data to get control totals
print("Loading CMS data for control totals...")
cms_trips = pd.read_csv(CMS_TRIP_PATH, low_memory=False)
cms_hh = pd.read_csv(CMS_HH_PATH, low_memory=False)

controls = calibration.get_cms_control_totals(cms_trips, cms_hh)
target_trips = controls['total_trips']
target_avg_trip_length = controls['avg_trip_length']

# Balance P/A totals in the zone_dict to match CMS control total
net.zone_dict = calibration.balance_productions_attractions(
    zone_dict=net.zone_dict,
    total_target_trips=target_trips
)

print("Production and attraction balancing complete.")

#### Step 5: Calibrate and Run Gravity Model

In [ ]:
print("
--- Step 5: Calibrating and Running Gravity Model ---")

# First, calculate the zone-to-zone OD distance matrix
print("Calculating zone-to-zone OD distance matrix...")
net.calc_zone_od_distance()

# Calibrate the gravity model to match the CMS average trip length
calibration_results = calibration.calibrate_gravity_model(
    net=net,
    target_avg_trip_length=target_avg_trip_length,
    trip_purpose=4, # Dummy purpose for total daily trips
    trip_rate_file=TRIP_RATE_TOTAL_PATH
)

final_demand = calibration_results['demand_df']
net.df_demand = final_demand # Update the demand df in the net object

print("Gravity model calibration and final run complete.")

#### Step 6: Save and Analyze Results

In [ ]:
print("
--- Step 6: Saving and Analyzing Results ---")

# Save the final mode-agnostic demand matrix and the new zone file
net.save_results_to_csv(
    demand=True,
    zone=True,
    demand_od_matrix=True,
    overwrite_file=True
)

print(f"Results saved to: {net.output_dir}")

print("
Final Demand Matrix Head:")
print(final_demand.head())

final_total_trips = final_demand['volume'].sum()
print(f"
Final total trips in OD matrix: {final_total_trips:,.0f}")
print(f"Target total trips from CMS:    {target_trips:,.0f}")
print(f"Difference: {final_total_trips - target_trips:,.0f} ({((final_total_trips - target_trips) / target_trips) * 100:.2f}%)")